In [1]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
llm = AzureChatOpenAI(
    model="gpt-35-turbo",
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-03-15-preview",
    temperature=0.5,
    azure_endpoint = os.getenv("AZURE_BASE_URL"),
)

In [3]:
# Agent for live search, will use open-source duckduckgo package 

# ! pip install -U duckduckgo-search

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

In [5]:
search = DuckDuckGoSearchRun()
search.invoke("Who won the medal in mens hockey in olympics 2024?")

"In a match that came as close as it gets, Netherlands men's hockey defeated Germany 1(3)-1(1) in an overtime shootout on Thursday, 8 August at the Olympic Games Paris 2024 to clinch their first Olympic gold medal in the men's competition since Sydney 2000.. It was almost as if they had a home-team advantage, as a sea of orange flooded the stadium with support for the Dutch, making for an ... The Indian men's hockey team won the bronze medal at the Paris 2024 Olympics.. India, who won a bronze medal at Tokyo 2020 to end a 41-year-long medal drought in hockey, secured their second successive medal in the sport at the Summer Games and 13th overall.Heading into Paris 2024, India were the most successful hockey team in Olympic history with eight gold, one silver and three bronze medals. The Netherlands won their first men's hockey Olympic gold medal in 24 years by beating Germany on penalties in a thrilling final at Paris 2024. The Netherlands won their first men's hockey Olympic ... Item 

In [7]:
# checking without agents
messages=[("system", "You are a helpful assistant."),
           ("human", "Who won the medal for mens hockey in olympics 2024?")]

llm.invoke(messages)

AIMessage(content="I'm sorry, but I am an AI language model and I cannot predict future events. The Olympics in 2024 have not yet occurred, so there is no information available on who will win the medal for men's hockey.", response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 34, 'total_tokens': 80}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-ba9626cb-8b0f-40c9-8243-053ab122e9bc-0', usage_metadata={'input_tokens': 34, 'output_tokens': 46, 'total_tokens': 80})

In [17]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.initialize import initialize_agent
from langchain.tools import Tool
from langchain.agents.agent_types import AgentType

In [18]:
search = DuckDuckGoSearchRun()

search_tools = Tool(
    name="DuckDuckGoSearch",
    func=search.run,
    description="Conducts a web search usong DuckDuckGo and provides Up-to-date information."
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [41]:
agent = initialize_agent(
    tools=[search_tools],
    llm=llm,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True,
    agent_type = AgentType.CONVERSATIONAL_REACT_DESCRIPTION
)

In [22]:
q1= "India's performance in Paris Olympics?"
agent.run(q1)



> Entering new AgentExecutor chain...
I need to search for information on India's performance in the Paris Olympics.
Action: DuckDuckGoSearch
Action Input: "India's performance in Paris Olympics"
Observation: Bronze medalist Swapnil Kusale of Team India poses with his medal in the 50m Rifle 3 Positions Men's Final on day six of the Olympic Games Paris 2024. India won a total of six medals -- one silver ... Here is a look at India's performance in athletics in Paris. Trending. 4. India consistently disappoints across several track events at Paris Olympics 2024 ... One of India's best performers at ... A 29-member Indian athletics team, headlined by Neeraj Chopra, competed at the Paris 2024 Olympics from August 1 to 9. Athletics kicked off on August 1 with race walking, the track and field events began on August 2.. Athletics at the Olympics included 48 medal events in all. They were held across four venues: Pont d'Iéna for race walking, Hôtel de Ville and Les Invalides for the start a

"India won a total of six medals, including a bronze in the 50m Rifle 3 Positions Men's Final and a bronze in men's hockey. The athletics team, headlined by Neeraj Chopra, competed in several track events but did not perform well. Overall, it was a campaign of mixed fortunes for India at the Paris Olympics."

In [23]:
q2= "Write an article based on performance of India in paris olympics based on the above info "
agent.run(q2)



> Entering new AgentExecutor chain...
I need to gather information on India's performance in the Paris Olympics
Action: DuckDuckGoSearch
Action Input: "India's performance in Paris Olympics"
Observation: Bronze medalist Swapnil Kusale of Team India poses with his medal in the 50m Rifle 3 Positions Men's Final on day six of the Olympic Games Paris 2024. India won a total of six medals -- one silver ... A 29-member Indian athletics team, headlined by Neeraj Chopra, competed at the Paris 2024 Olympics from August 1 to 9. Athletics kicked off on August 1 with race walking, the track and field events began on August 2.. Athletics at the Olympics included 48 medal events in all. They were held across four venues: Pont d'Iéna for race walking, Hôtel de Ville and Les Invalides for the start and ... Hockey: India beat Spain in the men's bronze medal match, their 13th Olympic podium. The Indian men's hockey team trumped Spain 2-1 in the bronze medal match at the Paris 2024 Olympics. As a resul

"India won a total of six medals at the Paris Olympics - one silver and five bronze. The Indian men's hockey team won a bronze medal, while Manu Bhaker became the first Indian woman to win a medal in shooting at the Olympics. India's Paralympic team also had a record haul of 29 medals - seven gold, nine silver, and 13 bronze. In athletics, a 29-member Indian contingent, including star athlete Neeraj Chopra, competed in several events. India's performance in the Paris Olympics was their third-best outing at the Summer Games after Tokyo 2020 and London 2012."

In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [44]:
decision_prompt= PromptTemplate(
    input_variables=["query"],
    template= """
    Query: {query}
    Determine if you know the answer to the shared query or it requires searching for it over the
    internet. Return "search", indicating the need to search the internet for an answer. If not required the answer the query yourself without taking internet's help"""

)
# decision_prompt | llm 
decision_chain = LLMChain(llm = llm, prompt = decision_prompt)

In [45]:
def should_use_tool(query):
    resp = decision_chain.run(query).strip().lower()
    print(resp)
    return resp.lower().strip()

def added_search(query):
  response_llm = should_use_tool(query)

  if (response_llm== "search"):
    search_results = search.run(query)
    print("Search Results: ", search_results)

    augmented_prompt = f"Based on the following search, answer the query: {query} \n\n {search_results}\n Answer: "
    final_answer= llm.invoke(augmented_prompt)
    return final_answer
    # return search_results

  else:
    response = response_llm
    return response

In [46]:
query_1="summarise India's performance in paris olympics in 100 words"
added_search(query_1)

search
Search Results:  One of India's best performers at the Paris Olympics 2024, Neeraj won the silver medal in the men's javelin throw finals with an 89.45m throw. With the silver in his pocket, Chopra brought home a ... Bronze medalist Swapnil Kusale of Team India poses with his medal in the 50m Rifle 3 Positions Men's Final on day six of the Olympic Games Paris 2024. India won a total of six medals -- one silver ... There are two ways to look at India's Paris Olympics 2024 report card. One is that the country has notched up its third-best performance at a single edition of the Games. The other is that a nation of 1.4 billion came 71st in the medal tally. August 13, 2024. The Paris Olympics 2024 ended with India winning a total of six medals: one silver and five bronze. This was a decrease from the seven medals, including one gold, that India achieved at the 2020 Tokyo Olympics. India sent a delegation of 117 athletes competing in 16 different sports. The performance showcased Indi

AIMessage(content="India won a total of six medals, one silver and five bronze, at the Paris Olympics 2024. Neeraj Chopra won the silver medal in the men's javelin throw finals with an 89.45m throw, while Swapnil Kusale won bronze in the 50m Rifle 3 Positions Men's Final. India's performance was its third-best at a single edition of the Games, but the country finished 71st in the medal tally. India sent a delegation of 117 athletes competing in 16 different sports. Manu Bhaker became the first Indian since independence to win multiple medals at the same edition, and Harmanpreet Singh was crucial to India winning a bronze medal.", response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 359, 'total_tokens': 504}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-eaee0bb4-0107-4d5b-9001-b741857b4a00-0', usage_metadata={'input_tokens': 359, 'output_tokens': 145, 'total_tokens': 5